# **Maestría en Inteligencia Artificial Aplicada**

## **Curso: Proyecto Integrador**

### Tecnológico de Monterrey

### Prof Dra. Grettel Barceló Alonso y Dr. Luis Eduardo Falcón Morales

## Avance II de Proyecto

## Ingeniería de Características

## Integrantes del Equipo:
### - Erika Cardona Rojas            A01749170
### - Miriam Bönsch                  A01330346
### - Mardonio Manuel Román Ramírez  A01795265

> # Generación de Nuevas Características

En esta etapa, realizaremos la estructuración y generación de nuevas variables basadas en el diseño experimental del estudio. El proceso se divide en dos ejes fundamentales:

- **Segmentación Temporal y de Grupo:** Identificación de los periodos de Intervención y Control para cada individuo, asignando etiquetas cronológicas (Pre y Post) a cada medición.
- **Derivación de Variables:** Con el fin de capturar la dinámica de respuesta al tratamiento, generaremos métricas de interacción. Esto incluye el cálculo de deltas ($\Delta$), tasas de cambio y variables booleanas (flags) de incremento, permitiendo una interpretación más profunda del efecto de los polifenoles en el rendimiento cognitivo.

In [1]:
# Librerias
import pandas as pd
import yaml
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from statsmodels.stats.multitest import multipletests

# Cargando Yaml
with open("../config.yaml", "r", encoding="utf-8") as file:
    config = yaml.safe_load(file)

import warnings

# Ignora solo los avisos de funciones que van a cambiar en el futuro
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Importando modulos
import sys
from pathlib import Path
# ----------------- Esto solo es para importar src de un folder antes
parent_folder = str(Path.cwd().parent)
if parent_folder not in sys.path:
    sys.path.append(parent_folder)

from src import functions as f
from src import modelsF as mf
#from src import models as m

In [2]:
# Cargando Base de Datos
df = pd.read_excel(r"../Data/Final_DF.xlsx")

In [3]:
# Creando columnas identificadoras
df['Treatment'] = np.where(df['tratamiento'] == 1 , 'Intervencion', 'Control')
df['Time'] = np.where(df['tiempo'] == 1 , 'Pre', 'Post')

# Eliminando otras variables identificadoras
df = df.drop(['visita', 'periodo', 'tiempo', 'tratamiento'], axis='columns')

> # Tecnicas De Filtrado

# Análisis: Modelo Lineal Mixto (LMM) y Difference-in-Differences (DiD)
Este análisis utiliza un **Modelo Lineal Mixto (Mixed Linear Model - LMM)**. Dado el diseño del ensayo clínico (mediciones repetidas en los mismos sujetos), este enfoque es superior a un ANOVA simple o pruebas t, ya que modela explícitamente la correlación intra-sujeto.

## Fundamento Teórico: Diferencia en Diferencias (DiD)

El objetivo central del estudio no es solo determinar si un grupo es mejor que otro en promedio, ni si los pacientes mejoran simplemente por el paso del tiempo. El objetivo es determinar si la **tasa de mejora** es distinta entre los grupos.

## Diseño
- N = 42 sujetos (ID)
- Entre-sujetos: `Group` (1 vs 2)
- Intra-sujeto (repetidas): 
  - `Treatment` (Intervencion vs Control)
  - `Time` (Pre vs Post)
- Cada sujeto aporta 4 observaciones: (Treatment × Time)

Este diseño induce correlación intra-sujeto que invalida pruebas independientes simples.

## Modelo
Para cada variable numérica Y:

Y ~ Group * Treatment * Time + (1 | ID)

Donde:
- `Group`, `Treatment`, `Time` son efectos fijos
- `(1|ID)` es intercepto aleatorio (captura heterogeneidad basal por sujeto)

## Interpretación DiD
- La interacción `Treatment:Time` representa el efecto Difference-in-Differences (cambio pre→post en Intervention menos cambio pre→post en Control) para el grupo de referencia.
- La interacción triple `Group:Treatment:Time` evalúa si el efecto DiD difiere entre Group 1 y Group 2.

Conceptualmente, buscamos la interacción estadística conocida como "Difference-in-Differences":

$$\text{Efecto Neto} = (\Delta_{\text{Grupo Experimental}}) - (\Delta_{\text{Grupo Control}})$$

$$\text{Efecto} = (\bar{X}_{A, \text{Final}} - \bar{X}_{A, \text{Inicio}}) - (\bar{X}_{B, \text{Final}} - \bar{X}_{B, \text{Inicio}})$$

## Salidas reportadas
- Coeficientes (betas), errores estándar, IC95%, p-values
- Ajuste por comparaciones múltiples (FDR) a través de variables (200+)
- Efectos marginales (contrastes) para estimar DiD por grupo

## Ventajas
- Modela explícitamente la estructura de medidas repetidas
- Usa toda la información sin colapsar datos a deltas
- Es más robusto que t-tests/ANOVA simple en presencia de correlación intra-sujeto

In [4]:
# Preparando el DataFrame
# Excluiremos las variables deltas, por la naturaleza de esas mediciones.
working_df, numeric_vars = f.prepare_dataframe(df, exclude_cols= config['Tratamiento_Variables']['Objetivos'][1:])

working_vars = ['id','grupo','Treatment','Time'] + numeric_vars

working_df = working_df[working_vars]
print(f"Realizaremos este primer análisis utilizando {len(numeric_vars) - 1} variables numéricas.")

Realizaremos este primer análisis utilizando 210 variables numéricas.


>> ## Ajuste de un LMM por variable

In [5]:
results, failed = mf.run_lmm_screen(
    working_df,
    id_col="id",
    group_col="grupo",
    treatment_col="Treatment",
    time_col="Time"
)

c:\Users\mardo\Documents\Repositories\Proyecto-Polifenoles\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:1634: UserWarning: Random effects covariance is singular
  warnings.warn(msg)
c:\Users\mardo\Documents\Repositories\Proyecto-Polifenoles\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2054: UserWarning: The random effects covariance matrix is singular.
  warnings.warn(_warn_cov_sing)
c:\Users\mardo\Documents\Repositories\Proyecto-Polifenoles\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
c:\Users\mardo\Documents\Repositories\Proyecto-Polifenoles\.venv\Lib\site-packages\statsmodels\regression\mixed_linear_model.py:2245: UserWarning: The random effects covariance matrix is singular.
  warnings.warn(_warn_cov_sing)
c:\Users\mardo\Documents\Repositories\Proyecto-Polifenoles\.venv\Lib\site-packages\sta

### Interpretando hallazgos

In [13]:
results.columns

Index(['variable', 'n', 'n_subjects', 'Triple_beta', 'Triple_p', 'DiD_beta',
       'DiD_p', 'Time_beta', 'Time_p', 'Treat_beta', 'Treat_p', 'converged',
       'Triple_q_FDR', 'DiD_q_FDR', 'Time_q_FDR', 'Treat_q_FDR'],
      dtype='object')

In [17]:
# Interpretando
df_interpretado = f.interpretar_hallazgos_final(results)

# Filtrar solo lo relevante para mostrar
cols_reporte = [
    "variable",                # Biomarcador
    "Interpretacion",          # Tu conclusión verbal (Lo más importante)
    
    # Magnitud del Efecto Clínico (Estimaciones Netas)
    "Efecto_Suplemento_G1",    # ¿Cuánto mejoró el Grupo 1?
    "Efecto_Suplemento_G2",    # ¿Cuánto mejoró el Grupo 2?
    
    # Desglose Estadístico (Evidencia)
    "Triple_q_FDR",            # ¿Es diferente entre grupos? (q-value ajustado)
    "DiD_q_FDR",               # ¿Es efecto del tratamiento? (q-value ajustado)
    
    # Componentes Crudos (Para referencia técnica si te preguntan)
    "Triple_beta",             # El diferencial puro
    "Variacion_Natural"        # El efecto tiempo/aprendizaje (Time_beta)
]

# Mostrar solo las variables donde hubo ALGÚN hallazgo (Prioridad 1 o 2)
hallazgos_significativos = df_interpretado[df_interpretado["Ranking"] < 3][cols_reporte]

print(f"Se encontraron {len(hallazgos_significativos)} biomarcadores con respuesta significativa.")

Se encontraron 26 biomarcadores con respuesta significativa.


In [16]:
hallazgos_significativos.to_excel('../Entregables/hallazgos_LMM.xlsx', index= False)